In [6]:
import gzip
from collections import defaultdict
import sklearn
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from time import time
from functools import reduce
from random import shuffle

In [7]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [8]:
data = [[l['userID'],l['businessID'],l['rating']] for l in readGz(".../train.json.gz")]
shuffle(data)
train_data = data[:180000]
test_data = data[180000:]

In [9]:
train_data_pd = DataFrame(train_data, columns=['userID','businessID','rating'])

In [10]:
def average(group):
    return sum(group.rating) / group.rating.count()
user_avg = train_data_pd.groupby("userID").apply(average).to_dict()
business_avg = train_data_pd.groupby("businessID").apply(average).to_dict()

In [11]:
user_business = train_data_pd.groupby("userID").apply(lambda group: list(group.businessID)).to_dict()
business_user = train_data_pd.groupby("businessID").apply(lambda group: list(group.userID)).to_dict()

In [9]:
default_avg = train_data_pd.rating.mean()

In [10]:
def prepare(data):
    X = []
    y = []
    for item in data:
        userID = item[0]
        businessID = item[1]
        try:
            user_avg_rating = user_avg[userID]
        except:
            user_avg_rating = default_avg
            
        try:
            business_avg_rating = business_avg[businessID]
        except:
            business_avg_rating = default_avg
            
        X.append([user_avg_rating, business_avg_rating])
        y.append(int(item[2]))
    return X, y

In [11]:
X_train, y_train = prepare(train_data)

In [12]:
X_test, y_test = prepare(test_data)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, Lasso, SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [15]:
clf_A = DecisionTreeRegressor()
clf_B = AdaBoostRegressor()
clf_C = RandomForestRegressor()
clf_D = LogisticRegression()
clf_E = Lasso()
clf_F = SGDRegressor()
clf_G = GradientBoostingRegressor()

In [17]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer

# TODO：初始化分类器
clf = GradientBoostingRegressor()

# TODO：创建你希望调节的参数列表
parameters = {'alpha':[0.2,0.6,0.8], 'learning_rate':[0.05, 0.1, 0.8]}

# TODO：创建一个fbeta_score打分对象
scorer = make_scorer(mean_squared_error)

In [18]:
# TODO：在分类器上使用网格搜索，使用'scorer'作为评价函数
grid_obj = GridSearchCV(clf, parameters, scoring=scorer, cv=None)

# TODO：用训练数据拟合网格搜索对象并找到最佳参数
grid_obj = grid_obj.fit(X_train, y_train)

# 得到estimator
best_clf = grid_obj.best_estimator_

In [19]:
best_clf


GradientBoostingRegressor(alpha=0.8, criterion='friedman_mse', init=None,
             learning_rate=0.8, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [20]:
clf_A.fit(X_train, y_train)
clf_B.fit(X_train, y_train)
clf_C.fit(X_train, y_train)
clf_D.fit(X_train, y_train)
clf_E.fit(X_train, y_train)
clf_F.fit(X_train, y_train)
clf_G.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [21]:
mean_squared_error(y_test,best_clf.predict(X_test))

0.63985692914728465

In [23]:
def prepare2(data):
    X = []
    for item in data:
        
        userID = item[0]
        businessID = item[1]
        try:
            user_avg_rating = user_avg[userID]
        except:
            user_avg_rating = default_avg

        try:
            business_avg_rating = business_avg[businessID]
        except:
            business_avg_rating = default_avg

        X.append([user_avg_rating, business_avg_rating])
    return X

X_test_online_raw = [[l[1], l[2]] for l in pd.read_csv("/Users/macbook/Downloads/assignment12/pairs_Rating.txt", delimiter="-").itertuples()]

X_test_online = prepare2(X_test_online_raw)
y_test_online = clf_G.predict(X_test_online)

In [54]:
predictions = open("/Users/macbook/Desktop/my_predictions_Rating.csv", 'w')
count = 0
for l in open("/Users/macbook/Downloads/assignment12/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    
    u,i = l.strip().split('-')
    predictions.write(u + '-' + i + ',' + str(y_test_online[count]) + '\n')
    count += 1

predictions.close()